In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1
%env ANYWIDGET_DEV=1

env: ANYWIDGET_HMR=1
env: ANYWIDGET_DEV=1


In [2]:
from seq import Widget

w = Widget()
w

True


Widget()

In [3]:
from datasets import load_dataset

ds = load_dataset("LawInformedAI/overruling")

ds

DatasetDict({
    train: Dataset({
        features: ['label', 'sentence1'],
        num_rows: 2400
    })
})

In [4]:
ds[0]

KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['train']"